## Sharing machine learning models

In [1]:
# Train a model.
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)
clr = RandomForestClassifier()
clr.fit(X_train, y_train)
clr.score(X_test, y_test) # accuracy on test data with trained model

0.9473684210526315

In [2]:
# Convert into ONNX format
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(clr, initial_types=initial_type)
with open("rf_iris.onnx", "wb") as f:
    f.write(onx.SerializeToString())

In [3]:
# Compute the prediction with ONNX Runtime
import onnxruntime as rt
import numpy

sess = rt.InferenceSession("rf_iris.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onx = sess.run([label_name], {input_name: X_test.astype(numpy.float32)})[0]

In [4]:
pred_onx

array([2, 2, 0, 2, 0, 2, 2, 2, 0, 0, 2, 2, 0, 0, 2, 2, 0, 1, 2, 2, 2, 1,
       1, 0, 2, 2, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2, 2, 0], dtype=int64)

In [8]:
from sklearn import metrics as score

score.accuracy_score(y_test, pred_onx) # accuracy on test data using loaded model

0.9473684210526315